# Intuition

Randomly set neurons in the network to 0 (and the neurons detlas for back prop). Like the clay analogy , imagine now the clay is sticky rocks the size of dimes. Each only captures partial impressions of the shape won't latch on to insignificant details - averaging of the shape. 

100 neural networks, will find the same broad signial, but different parts of the  noise. 

# Math

In [5]:
import numpy as np

dropout_mask = np.random.randint(2,size = 10) #50% random 1/0, randint(how many randoms, size )
dropout_mask

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0])

In [76]:
dropout_mask = np.random.randint(2,size = (2,8)) #50% random 1/0
dropout_mask

array([[1, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1]])

In [77]:
sl = np.array([[2,2,2],
              [2,2,2],
              [2,2,2],
              [2,2,2]])

w01 = 2*np.random.random((3,3)) - 1
layer_1 = np.dot(sl, w01)

dropout_mask = np.random.randint(2, size = layer_1.shape)
dropout_mask

array([[1, 0, 0],
       [1, 1, 0],
       [0, 0, 1],
       [0, 1, 0]])

In [78]:
layer_1

array([[ 0.49234719, -2.24365296,  0.01728756],
       [ 0.49234719, -2.24365296,  0.01728756],
       [ 0.49234719, -2.24365296,  0.01728756],
       [ 0.49234719, -2.24365296,  0.01728756]])

In [79]:
np.dot(layer_1, dropout_mask) 

ValueError: shapes (4,3) and (4,3) not aligned: 3 (dim 1) != 4 (dim 0)

In [80]:
layer_1 *= dropout_mask * 2 #element wise multiplication - not weighted sum as above. 
layer_1

array([[ 0.98469438, -0.        ,  0.        ],
       [ 0.98469438, -4.48730592,  0.        ],
       [ 0.        , -0.        ,  0.03457512],
       [ 0.        , -4.48730592,  0.        ]])

# MNIST IT

In [6]:
import sys
import numpy as np
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(f"x_train:{x_train.shape}, y_train:{y_train.shape}") #60k
print(f"x_test:{x_test.shape}, y_test:{y_test.shape}") #10k

def one_hot_labels(digit_labels):
    
    one_hot_labels = np.zeros((len(digit_labels),10))

    for i,l in enumerate(digit_labels):
        one_hot_labels[i][l] = 1
    
    return(one_hot_labels)

relu = lambda x:(x>=0) * x # returns x if x > 0, return 0 otherwise
relu2deriv = lambda x: x>=0 # returns 1 for input > 0, return 0 otherwise

x_train:(60000, 28, 28), y_train:(60000,)
x_test:(10000, 28, 28), y_test:(10000,)


In [7]:
trn_images = x_train[0:1000].reshape(1000, 28*28)/255
trn_labels = one_hot_labels(y_train[0:1000])

tst_images = x_test.reshape(len(x_test), 28*28)/255
tst_labels = one_hot_labels(y_test)

np.random.seed(1)
alpha, iterations, hidden_size, pixels_per_image, num_labels = (0.005, 350, 40, 784, 10)

In [83]:
%time
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    
    for i in range(len(trn_images)):
        layer_0 = trn_images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        
        ###########################################
        dropout_mask = np.random.randint(2, size = layer_1.shape)
        layer_1 *= dropout_mask * 2 #amplification
        ############################################
        
        layer_2 = np.dot(layer_1,weights_1_2)

        error += np.sum((trn_labels[i:i+1] - layer_2) ** 2)
        correct_cnt += int(np.argmax(layer_2) == \
                                        np.argmax(trn_labels[i:i+1]))

        #spread the delta
        layer_2_delta = (trn_labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)* relu2deriv(layer_1)
        
        ##############################
        layer_1_delta *= dropout_mask
        ##############################
        
        
        #input* delta
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
        
    #print(f"Iteration:{j}, Trn-Err:.{error/len(trn_images):.3f},Trn-Acc:{correct_cnt/float(len(trn_images)):.3f}", end = '\r')
    
    
    if(j % 50 == 0 or j == iterations-1):
        tst_error, tst_correct_cnt = (0.0, 0)

        for i in range(len(tst_images)):

            layer_0 = tst_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1,weights_1_2)

            tst_error += np.sum((tst_labels[i:i+1] - layer_2) ** 2)
            tst_correct_cnt += int(np.argmax(layer_2) == np.argmax(tst_labels[i:i+1]))
            
        msg = (
            f"Iteration:{j}, "
            f"Trn-Err:.{error/len(trn_images):.3f}, "
            f"Tst-Err:.{tst_error/len(tst_images):.3f}, "
            f"Trn-Acc:{correct_cnt/float(len(trn_images)):.3f}, "
            f"Tst-Acc:{tst_correct_cnt/float(len(tst_images)):.3f}."
        )
        print(msg)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
Iteration:0, Trn-Err:.0.885, Tst-Err:.0.718, Trn-Acc:0.289, Tst-Acc:0.542.
Iteration:50, Trn-Err:.0.463, Tst-Err:.0.431, Trn-Acc:0.742, Tst-Acc:0.815.
Iteration:100, Trn-Err:.0.453, Tst-Err:.0.433, Trn-Acc:0.769, Tst-Acc:0.804.
Iteration:150, Trn-Err:.0.458, Tst-Err:.0.447, Trn-Acc:0.783, Tst-Acc:0.792.
Iteration:200, Trn-Err:.0.443, Tst-Err:.0.437, Trn-Acc:0.796, Tst-Acc:0.803.
Iteration:250, Trn-Err:.0.433, Tst-Err:.0.421, Trn-Acc:0.789, Tst-Acc:0.805.
Iteration:300, Trn-Err:.0.408, Tst-Err:.0.434, Trn-Acc:0.804, Tst-Acc:0.795.
Iteration:349, Trn-Err:.0.419, Tst-Err:.0.421, Trn-Acc:0.802, Tst-Acc:0.808.


# Batch Updates

In [56]:
trn_images.shape

(1000, 784)

Key idea is to train on batch and update the weights then. 

In [9]:
alpha, iterations, hidden_size, pixels_per_image, num_labels = (0.001, 1, 40, 28*28, 10)
batch_size = 100

weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    
    #################################################
    for i in range(int(len(trn_images) / batch_size)):
        batch_start, batch_end = ((i * batch_size),((i+1)*batch_size))
        print(f"batch_start:{batch_start},batch_end:{batch_end}")
    
        layer_0 = trn_images[batch_start:batch_end]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        
        print(f"layer_0:{layer_0.shape},w01{weights_0_1.shape}, layer_1:{layer_1.shape}")

batch_start:0,batch_end:100
layer_0:(100, 784),w01(784, 40), layer_1:(100, 40)
batch_start:100,batch_end:200
layer_0:(100, 784),w01(784, 40), layer_1:(100, 40)
batch_start:200,batch_end:300
layer_0:(100, 784),w01(784, 40), layer_1:(100, 40)
batch_start:300,batch_end:400
layer_0:(100, 784),w01(784, 40), layer_1:(100, 40)
batch_start:400,batch_end:500
layer_0:(100, 784),w01(784, 40), layer_1:(100, 40)
batch_start:500,batch_end:600
layer_0:(100, 784),w01(784, 40), layer_1:(100, 40)
batch_start:600,batch_end:700
layer_0:(100, 784),w01(784, 40), layer_1:(100, 40)
batch_start:700,batch_end:800
layer_0:(100, 784),w01(784, 40), layer_1:(100, 40)
batch_start:800,batch_end:900
layer_0:(100, 784),w01(784, 40), layer_1:(100, 40)
batch_start:900,batch_end:1000
layer_0:(100, 784),w01(784, 40), layer_1:(100, 40)


# Parameter Alterations

- batch size 100 introduced. 
- hidden size increased from 40 to 100
- alpha reduced from 0.005 to 0.001
- iterations reduced from 350 to 300 

In [11]:
np.random.seed(1)

alpha, iterations, hidden_size, pixels_per_image, num_labels = (0.001, 300, 100, 28*28, 10)
batch_size = 500

weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    
    #################################################
    for i in range(int(len(trn_images) / batch_size)):
        batch_start, batch_end = ((i * batch_size),((i+1)*batch_size))
        
        layer_0 = trn_images[batch_start:batch_end]
        ###########################################
        
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        
        layer_2 = np.dot(layer_1,weights_1_2)
        

        error += np.sum((trn_labels[batch_start:batch_end] - layer_2) ** 2)
        print(f"layer_0:{layer_0.shape},layer_1:{layer_1.shape}, layer_2:{layer_2.shape}")
        
        layer_2_delta = (trn_labels[batch_start:batch_end]-layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)* relu2deriv(layer_1)
        layer_1_delta *= dropout_mask
        print(f"layer_2_deleta:{layer_2_delta.shape},layer_1_delta:{layer_1_delta.shape}")

        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
        
        
        #••••••••••••••••••••••••••••••••••••••••••••••
        for k in range(batch_size):
            correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(trn_labels[batch_start+k:batch_start+k+1]))
        #**********************************************
        
    if(j % 10 == 0 or j == iterations-1):
        tst_error, tst_correct_cnt = (0.0, 0)

        for i in range(len(tst_images)):

            layer_0 = tst_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1,weights_1_2)

            tst_error += np.sum((tst_labels[i:i+1] - layer_2) ** 2)
            tst_correct_cnt += int(np.argmax(layer_2) == np.argmax(tst_labels[i:i+1]))
            
        msg = (
            f"Iteration:{j}, "
            f"Trn-Err:.{error/len(trn_images):.3f}, "
            f"Tst-Err:.{tst_error/len(tst_images):.3f}, "
            f"Trn-Acc:{correct_cnt/float(len(trn_images)):.3f}, "
            f"Tst-Acc:{tst_correct_cnt/float(len(tst_images)):.3f}."
        )
        print(msg)
        
        

            

layer_0:(500, 784),layer_1:(500, 100), layer_2:(500, 10)
layer_2_deleta:(500, 10),layer_1_delta:(500, 100)
layer_0:(500, 784),layer_1:(500, 100), layer_2:(500, 10)
layer_2_deleta:(500, 10),layer_1_delta:(500, 100)
Iteration:0, Trn-Err:.3.619, Tst-Err:.1.285, Trn-Acc:0.096, Tst-Acc:0.219.
layer_0:(500, 784),layer_1:(500, 100), layer_2:(500, 10)
layer_2_deleta:(500, 10),layer_1_delta:(500, 100)
layer_0:(500, 784),layer_1:(500, 100), layer_2:(500, 10)
layer_2_deleta:(500, 10),layer_1_delta:(500, 100)
layer_0:(500, 784),layer_1:(500, 100), layer_2:(500, 10)
layer_2_deleta:(500, 10),layer_1_delta:(500, 100)
layer_0:(500, 784),layer_1:(500, 100), layer_2:(500, 10)
layer_2_deleta:(500, 10),layer_1_delta:(500, 100)
layer_0:(500, 784),layer_1:(500, 100), layer_2:(500, 10)
layer_2_deleta:(500, 10),layer_1_delta:(500, 100)
layer_0:(500, 784),layer_1:(500, 100), layer_2:(500, 10)
layer_2_deleta:(500, 10),layer_1_delta:(500, 100)
layer_0:(500, 784),layer_1:(500, 100), layer_2:(500, 10)
layer_2_dele